In [ ]:
import pandas as pd
import json
import glob
import numpy as np 

In [ ]:
import re

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sys
import os
from pathlib import Path
import yaml


# Adjust the path to your project root folder
project_root = os.path.abspath(
    os.path.join("..")
)  # from notebooks/ up one level

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_loading.data_loading.data_loader import load_data_from_json
from src.data_loading.preprocessing.preprocessing import preprocess_df
from src.data_loading.preprocessing.imputation import impute_missing_values


# go two levels up from notebook dir -> project root
ROOT = (
    Path(__file__).resolve().parents[2]
    if "__file__" in globals()
    else Path.cwd().parents[1]
)
CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "preprocessing_config.yaml"
)

with open(CONFIG_PATH) as f:
    CONFIG = yaml.safe_load(f)

df_raw = load_data_from_json("../data/parsed_json/*.json")
df_clean = preprocess_df(
    df_raw,
    drop_raw=CONFIG["preprocessing"]["drop_raw"],
    numeric_cols=CONFIG["preprocessing"]["numeric_cols"],
)
df_clean = impute_missing_values(
    df_clean, CONFIG["preprocessing"]["imputation"]
)
# Drop price_num NaNs for the training of the model
# df_clean = df_clean[df_clean["price_num"].notna()]
df = df_clean.copy()

In [ ]:
# import os
# import shutil
# import json

# new_folder = "../data/parsed_json"
# old_folder = "../data/parsed_json_old"

# def load_address_to_file(folder):
#     mapping = {}
#     for filename in os.listdir(folder):
#         if not filename.endswith(".json"):
#             continue
#         path = os.path.join(folder, filename)
#         try:
#             with open(path, "r", encoding="utf-8") as f:
#                 data = json.load(f)
#                 if "address" in data:
#                     mapping[data["address"]] = path
#         except Exception as e:
#             print(f"⚠️ Error reading {path}: {e}")
#     return mapping

# # Map addresses to file paths
# new_map = load_address_to_file(new_folder)
# old_map = load_address_to_file(old_folder)

# # Find addresses missing in new scrape
# missing_addresses = set(old_map.keys()) - set(new_map.keys())

# print(f"Found {len(missing_addresses)} properties missing in new scrape.")

# # Copy them into new folder
# for addr in missing_addresses:
#     old_file = old_map[addr]
#     filename = os.path.basename(old_file)
#     new_file = os.path.join(new_folder, filename)

#     if not os.path.exists(new_file):  # avoid overwriting
#         shutil.copy2(old_file, new_file)
#         print(f"➕ Added {filename} for {addr}")


In [ ]:
df

In [ ]:
df.replace("N/A", pd.NA, inplace=True)

Will refactor below code

In [ ]:
# df["bedrooms_num"] = pd.to_numeric(df["bedrooms"], errors="coerce")
# df["nr_rooms_num"] = pd.to_numeric(df["nr_rooms"], errors="coerce")
# df["bathrooms_num"] = pd.to_numeric(df["bathrooms"], errors="coerce")
# df["year_of_construction_num"] = pd.to_numeric(
#     df["year_of_construction"], errors="coerce"
# )

df[
    [
        "price_num",
        "size_num",
        "bedrooms",
        "nr_rooms",
        "bathrooms",
        "year_of_construction_num",
    ]
].describe()

In [ ]:
# # Price Distribution
# sns.histplot(df["price_num"].dropna(), bins=15)
# plt.title("Price Distribution")
# plt.xlabel("Price (€)")
# # Adjust x-axis ticks
# max_price = df["price_num"].max()
# min_price = df["price_num"].min()

# step = 5000000 # set step size (e.g., €50k)
# plt.xticks(np.arange(min_price, max_price + step, step))
# plt.show()

import matplotlib.ticker as ticker

sns.histplot(df["price_num"].dropna(), bins=15)
plt.title("Price Distribution")
plt.xlabel("Price (€)")

# Format ticks as euros with "k"
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(500000))  # step = 500k
ax.xaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x, _: f"€{int(x/1000)}k")
)
plt.xticks(rotation=45, ha="right")
plt.show()

# Size Distribution
sns.histplot(df["size_num"].dropna(), bins=15)
plt.title("Size Distribution")
plt.xlabel("Size (m²)")
plt.show()

# Bedrooms Count
sns.countplot(x="bedrooms", data=df)
plt.title("Bedrooms Count")
plt.show()

# Year of construction histogram
sns.histplot(df["year_of_construction_num"].dropna(), bins=10)
plt.title("Year of Construction")
plt.show()

In [ ]:
# Neighborhood Counts
print(df["neighborhood"].value_counts())
# Energy label counts
print(df["energy_label"].value_counts())

In [ ]:
# Facilities unique values (might be comma separated string)
print(df["facilities"].dropna().head())

# Missing values
print(df.isna().sum())

#### 0. Missing values


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df.isna(), cbar=False)
plt.title("Missing Values Heatmap")
plt.show()

print(df.isna().sum().sort_values(ascending=False))

#### 1. Correlation Analysis of Numeric Features


In [ ]:
df.corr(numeric_only=True)["price_num"].sort_values(ascending=False)

In [ ]:
corr = df.corr(numeric_only=True)
print(corr.columns)

In [ ]:
plt.scatter(df["size_num"], df["price_num"])
plt.xlabel("Living Area (m²)")
plt.ylabel("Price (€)")
plt.title("Price vs Living Area")
plt.show()

In [ ]:
numeric_cols = [
    "price_num",
    "size_num",
    "bedrooms",
    "nr_rooms",
    "bathrooms",
    "year_of_construction_num",
]

corr = df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix of Numeric Features")
plt.show()

#### 2. Price per m² Calculation & Analysis

Derive price per m² from price and size, check its distribution and relation to neighborhood average


In [ ]:
df["price_per_m2"] = df["price_num"] / df["size_num"]

sns.histplot(df["price_per_m2"].dropna(), bins=20)
plt.title("Price per m² Distribution")
plt.xlabel("Price per m² (€)")
plt.show()

In [ ]:
df.columns

In [ ]:
sns.scatterplot(x="price_per_m2_neighborhood", y="price_per_m2", data=df)
plt.title("Listing Price/m² vs Neighborhood Avg Price/m²")
plt.xlabel("Neighborhood Avg Price/m² (€)")
plt.ylabel("Listing Price/m² (€)")
plt.show()

#### 3. Outlier Detection in Price or Price per m²

Visualize and identify listings with suspiciously high or low prices


In [ ]:
Q1 = df["price_num"].quantile(0.25)
Q3 = df["price_num"].quantile(0.75)

IQR = Q3 - Q1

outliers = df[
    (df["price_num"] < (Q1 - 1.5 * IQR)) | (df["price_num"] > (Q3 + 1.5 * IQR))
]

print(f"Found {len(outliers)} price outliers")
print(outliers[["address", "price_num"]])

#### 4. Feature Engineering: Extract Useful Info from Text

Example: Count number of facilities


In [ ]:
df["num_facilities"] = df["facilities"].apply(
    lambda x: len(x.split(",")) if pd.notna(x) else 0
)
sns.histplot(df["num_facilities"], bins=10)
plt.title("Distribution of Number of Facilities")
plt.show()

#### 5. Explore Missing Data Patterns


In [ ]:
import missingno as msno

msno.matrix(df)
plt.show()

msno.bar(df)
plt.show()

#### 6. Categorical Analysis

Distribution of energy labels or balconies


In [ ]:
sns.countplot(x="energy_label", data=df)
plt.title("Energy Label Distribution")
plt.show()

sns.countplot(x="balcony", data=df)
plt.title("Balcony Availability")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df

#### Non linear relationships

In [ ]:
import statsmodels.api as sm
import numpy as np

# Drop NA
df_clean = df[["size_num", "price_num"]].dropna()

# Fit LOWESS
lowess = sm.nonparametric.lowess
z = lowess(df_clean["price_num"], df_clean["size_num"], frac=0.3)

# Plot
plt.scatter(df_clean["size_num"], df_clean["price_num"], s=20, alpha=0.5)
plt.plot(z[:,0], z[:,1], color="red", linewidth=2)
plt.title("Price vs Size with LOWESS smoothing")
plt.xlabel("Size (m²)")
plt.ylabel("Price (€)")
plt.show()


#### Feature Importance via Tree Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
df = df.dropna(subset=["price_num"])
features = ["size_num", "bedrooms", "year_of_construction_num", "nr_rooms"]
X = df[features].fillna(0)
y = df["price_num"]

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X, y)
importances = pd.Series(rf.feature_importances_, index=features)
print(importances.sort_values(ascending=False))


#### Neighborhood clusters

In [ ]:
# Extract first 3 digits of postal code
df["postcode_area"] = df["postal_code_clean"].str[:3]


postcode_stats = df.groupby("postcode_area").agg({
    "price_num": "mean",
    "size_num": "mean",
    "nr_rooms": "mean",
    "bedrooms": "mean",
    "address": "count"  # number of listings
}).rename(columns={"address": "num_listings"})


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
features_scaled = scaler.fit_transform(postcode_stats)

kmeans = KMeans(n_clusters=4, random_state=42)
postcode_stats["cluster"] = kmeans.fit_predict(features_scaled)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.scatterplot(
    x=postcode_stats["size_num"],
    y=postcode_stats["price_num"],
    hue=postcode_stats["cluster"],
    palette="Set2",
    s=100
)

for i, name in enumerate(postcode_stats.index):
    plt.text(
        postcode_stats["size_num"][i],
        postcode_stats["price_num"][i],
        name,
        fontsize=8,
        alpha=0.7
    )

plt.title("Postcode Area Clusters by Avg Price & Size")
plt.xlabel("Average Size (m²)")
plt.ylabel("Average Price (€)")
plt.show()


#### Now let's use amsterdam map and pc4

In [ ]:
postcode4_stats = df.groupby("postal_code_clean").agg({
    "price_num": "mean",
    "size_num": "mean",
    "nr_rooms": "mean",
    "bedrooms": "mean",
    "address": "count"  # number of listings
}).rename(columns={"address": "num_listings"})

# Optional: filter low-count PC4s
postcode4_stats = postcode4_stats[postcode4_stats["num_listings"] >= 3]



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
features_scaled = scaler.fit_transform(postcode4_stats)

kmeans = KMeans(n_clusters=5, random_state=42)
postcode4_stats["cluster"] = kmeans.fit_predict(features_scaled)


In [ ]:
import geopandas as gpd
# Load PC4 shapefile (replace path with your extracted .shp file)
gdf = gpd.read_file("../data/georef-netherlands-postcode-pc4/georef-netherlands-postcode-pc4.shp")

# Quick check
print(gdf.head())
print(gdf.crs)  # check coordinate system



In [ ]:
postcode4_stats = postcode4_stats.copy()
postcode4_stats["pc4_code"] = postcode4_stats.index.str[:4]
postcode4_stats.set_index("pc4_code", inplace=True)

In [ ]:
# # Merge clusters into the GeoDataFrame using the correct column
# gdf = gdf.merge(postcode4_stats[["cluster"]], left_on="pc4_code", right_index=True, how="left")

# # Check the merge
# print(gdf[["pc4_code", "cluster"]].head())


In [ ]:
# Make a copy to be safe
postcode4_stats = postcode4_stats.copy()

# If your index is full postal_code_clean like '1011 DG'
postcode4_stats["pc4_code"] = postcode4_stats.index.str[:4]

# Set pc4_code as index for merging
postcode4_stats.set_index("pc4_code", inplace=True)

# Optional: check
print(postcode4_stats.head())


In [ ]:
# # Merge clusters into GeoDataFrame
# gdf = gdf.merge(postcode4_stats[["cluster"]], left_on="pc4_code", right_index=True, how="left")

# # Check merge
# print(gdf[["pc4_code", "cluster"]].head(10))


In [ ]:
missing = set(gdf["pc4_code"]) - set(postcode4_stats.index)
print("PC4s in shapefile but not in dataset:", missing)


In [ ]:
# Merge cluster and stats into GeoDataFrame
gdf = gdf.merge(
    postcode4_stats[["cluster", "price_num", "size_num", "num_listings"]],
    left_on="pc4_code",
    right_index=True,
    how="left"
)

# Replace NaN clusters with -1 for visualization
gdf["cluster_fill"] = gdf["cluster"].fillna(-1)


In [ ]:
gdf

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,12))

# Plot boundaries
gdf.boundary.plot(ax=ax, linewidth=0.5, color='black')

# Plot clusters, with NaNs in light gray
gdf.plot(column='cluster', ax=ax, legend=True, cmap='Set2', 
         legend_kwds={'title': "Cluster"},
         missing_kwds={"color": "lightgrey", "label": "No data"})

plt.title("Amsterdam Postcode Area Clusters (PC4)")
plt.axis('off')  # optional
plt.show()


In [ ]:
import branca.colormap as cm

# Define colors for clusters, including NaN as light gray
cluster_colors = {
    -1: "lightgray",  # no data
    0: "#1f77b4",
    1: "#ff7f0e",
    2: "#2ca02c",
    3: "#d62728",
    4: "#9467bd"
}

# Function to map cluster to color
def style_function(feature):
    cluster = feature["properties"]["cluster_fill"]
    return {
        "fillColor": cluster_colors.get(cluster, "lightgray"),
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7
    }


In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Center map on Amsterdam
m = folium.Map(location=[52.3702, 4.8952], zoom_start=12)

# Add GeoJson layer with clusters and tooltips
tooltip = GeoJsonTooltip(
    fields=["pc4_code", "cluster", "price_num", "size_num", "num_listings"],
    aliases=["PC4:", "Cluster:", "Avg Price (€):", "Avg Size (m²):", "Listings:"],
    localize=True
)

folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=tooltip,
    name="PC4 Clusters"
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display map
m


In [ ]:
# Save interactive map to an HTML file
m.save("amsterdam_clusters_map.html")


#### With Elbow method selecting clusters

Too little clusters now

In [ ]:
from sklearn.preprocessing import StandardScaler

# Choose features for clustering
features = ["price_num", "size_num", "nr_rooms", "bedrooms"]

# Fill missing values with 0 (or consider median imputation)
X = postcode4_stats[features].fillna(0)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
K = range(2, 15)  # test 2 to 14 clusters

for k in K:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

# Plot the elbow
plt.figure(figsize=(8,5))
plt.plot(K, inertia, marker='o')
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal k")
plt.show()


In [ ]:
# ==============================
# 1️⃣ Imports
# ==============================
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import folium
from folium.features import GeoJsonTooltip
from branca.element import Template, MacroElement
from shapely.geometry import Point

# ==============================
# 2️⃣ Load shapefile and dataset
# ==============================
gdf = gpd.read_file("../data/georef-netherlands-postcode-pc4/georef-netherlands-postcode-pc4.shp")
gdf["pc4_code"] = gdf["pc4_code"].astype(str).str.strip()

df["pc4_code"] = df["postal_code_clean"].str[:4]

# ==============================
# 3️⃣ Aggregate stats per PC4
# ==============================
postcode4_stats = df.groupby("pc4_code").agg(
    price_num=("price_num", "mean"),
    size_num=("size_num", "mean"),
    nr_rooms=("nr_rooms", "mean"),
    bedrooms=("bedrooms", "mean"),
    num_listings=("price_num", "count")
).reset_index().set_index("pc4_code")

postcode4_stats["price_per_m2"] = postcode4_stats["price_num"] / postcode4_stats["size_num"]

# ==============================
# 4️⃣ KMeans Clustering
# ==============================
features = ["price_per_m2", "size_num", "nr_rooms", "bedrooms"]
X = postcode4_stats[features].fillna(0)
X_scaled = StandardScaler().fit_transform(X)

k = 10
kmeans = KMeans(n_clusters=k, random_state=42)
postcode4_stats["cluster"] = kmeans.fit_predict(X_scaled)

# ==============================
# 5️⃣ Merge into GeoDataFrame
# ==============================
postcode4_stats.index = postcode4_stats.index.str.strip()
gdf = gdf.merge(
    postcode4_stats[["cluster", "price_num", "size_num", "num_listings", 
                     "price_per_m2", "nr_rooms", "bedrooms"]],
    left_on="pc4_code",
    right_index=True,
    how="left"
)

gdf["cluster_fill"] = gdf["cluster"].fillna(-1)

# ==============================
# 6️⃣ Neighborhood context
# ==============================
amsterdam_center = Point(4.8952, 52.3702)
gdf = gdf.to_crs(epsg=4326)  # ensure WGS84

gdf["centroid"] = gdf.geometry.centroid
gdf["dist_to_center_km"] = gdf["centroid"].distance(amsterdam_center) * 111  # approx km
gdf["area_km2"] = gdf.geometry.to_crs(epsg=3857).area / 1e6
gdf["listing_density"] = gdf["num_listings"] / gdf["area_km2"]

# ==============================
# 7️⃣ Color by cluster
# ==============================
cluster_colors = {-1: "lightgray"}
palette = ["#1f77b4","#ff7f0e","#2ca02c","#d62728","#9467bd",
           "#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"]
for i in range(k):
    cluster_colors[i] = palette[i % len(palette)]

def style_function(feature):
    cluster = feature["properties"]["cluster_fill"]
    return {
        "fillColor": cluster_colors.get(cluster, "lightgray"),
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7
    }

# ==============================
# 8️⃣ Folium map with tooltip
# ==============================
m = folium.Map(location=[52.3702, 4.8952], zoom_start=12)

tooltip = GeoJsonTooltip(
    fields=["pc4_code", "cluster", "price_num", "size_num", "price_per_m2", 
            "num_listings", "nr_rooms", "bedrooms", "dist_to_center_km", "listing_density"],
    aliases=["PC4:", "Cluster:", "Avg Price (€):", "Avg Size (m²):", "€/m²:", 
             "Listings:", "Avg Rooms:", "Avg Bedrooms:", "Distance to Center (km):", "Listings/km²:"],
    localize=True
)

# Keep only columns needed for GeoJson/tooltip
gdf_for_map = gdf.drop(columns=["centroid"])

folium.GeoJson(
    gdf_for_map,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)

# ==============================
# Legend: Rounded ranges + categories
# ==============================
# Price per m² categories
price_bins = [0, 6000, 9000, 15000]  # €/m²
price_labels = ["Low", "Medium", "High"]

# Function to categorize
def categorize(val, bins, labels):
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            return labels[i]
    return labels[-1]

# Compute cluster stats
cluster_stats = gdf.groupby("cluster_fill").agg(
    price_m2_min=("price_per_m2", "min"),
    price_m2_max=("price_per_m2", "max"),
    size_min=("size_num", "min"),
    size_max=("size_num", "max")
)

legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 280px; 
    z-index:9999; font-size:14px;
    background-color:white; padding: 10px; border:2px solid grey;
">
<b>Cluster Legend</b><br>
<i style="background:lightgray;width:12px;height:12px;display:inline-block;"></i> No data<br>
"""

for i in range(k):
    stats = cluster_stats.loc[i]
    if stats.isnull().any():
        legend_html += f'<i style="background:{palette[i % len(palette)]};width:12px;height:12px;display:inline-block;"></i> Cluster {i}<br>'
        continue

    # Rounded values
    price_min_r = int(stats["price_m2_min"]//100*100)
    price_max_r = int(stats["price_m2_max"]//100*100)
    size_min_r = int(stats["size_min"]//5*5)
    size_max_r = int(stats["size_max"]//5*5)

    price_cat = categorize((price_min_r + price_max_r)/2, price_bins, price_labels)

    legend_html += (
        f'<i style="background:{palette[i % len(palette)]};width:12px;height:12px;display:inline-block;"></i> '
        f'Cluster {i} ({price_cat} €/m², {price_min_r}-{price_max_r} €/m², size {size_min_r}-{size_max_r} m²)<br>'
    )

legend_html += "</div>{% endmacro %}"

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ==============================
# Save map
# ==============================
m.save("amsterdam_clusters_neighborhood_map.html")


In [ ]:
# ==============================
# 1️⃣ Cluster Summary Table
# ==============================
cluster_summary = gdf.groupby("cluster_fill").agg(
    pc_count=("pc4_code", "count"),
    price_min=("price_num", "min"),
    price_max=("price_num", "max"),
    price_avg=("price_num", "mean"),
    size_min=("size_num", "min"),
    size_max=("size_num", "max"),
    size_avg=("size_num", "mean"),
    price_per_m2_min=("price_per_m2", "min"),
    price_per_m2_max=("price_per_m2", "max"),
    price_per_m2_avg=("price_per_m2", "mean"),
    dist_to_center_avg=("dist_to_center_km", "mean"),
    listing_density_avg=("listing_density", "mean")
).reset_index()

# Round numeric columns for readability
numeric_cols = cluster_summary.select_dtypes(include="number").columns
cluster_summary[numeric_cols] = cluster_summary[numeric_cols].round(0)

# Sort clusters by avg price_per_m2 descending
cluster_summary = cluster_summary.sort_values("price_per_m2_avg", ascending=False)

# Optional: categorize clusters by €/m²
def price_category(p):
    if p < 6000:
        return "Low €/m²"
    elif p < 10000:
        return "Medium €/m²"
    else:
        return "High €/m²"

cluster_summary["price_category"] = cluster_summary["price_per_m2_avg"].apply(price_category)

# ==============================
# 2️⃣ Display Summary Table
# ==============================
print(cluster_summary[[
    "cluster_fill", "price_category", "price_per_m2_min", "price_per_m2_max",
    "size_min", "size_max", "dist_to_center_avg", "listing_density_avg", "pc_count"
]])

# ==============================
# 3️⃣ Optional: Visualization
# ==============================
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.barplot(data=cluster_summary, x="cluster_fill", y="price_per_m2_avg", hue="price_category", dodge=False)
plt.title("Average Price per m² by Cluster")
plt.xlabel("Cluster")
plt.ylabel("€/m²")
plt.legend(title="Category")
plt.show()

# Similarly, you can create plots for avg size, distance to center, or listing density
